In [ ]:
# === 데이터 처리 및 기본 라이브러리 ===
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()
import gc

# === 머신러닝 모델 및 검증 도구 ===
import lightgbm as lgb
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score

# === 이상치 탐지 ===
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

# === 특징 추출(Feature Engineering) 도구 ===
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

# === 딥러닝(임베딩, PPL) 모델 ===
import torch
from transformers import AutoTokenizer, AutoModel

# === 병렬 처리를 위한 라이브러리 ===
from joblib import Parallel, delayed



In [ ]:
# --- GPU 설정 및 최적화 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# TF32 최적화 활성화
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print("TF32 최적화 활성화됨")

def print_gpu_utilization():
    """GPU 메모리 사용량 모니터링"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        max_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"GPU 메모리 사용량: {allocated:.2f} GB")
        print(f"GPU 메모리 캐시: {reserved:.2f} GB")
        print(f"GPU 메모리 사용률: {allocated/max_memory*100:.1f}%")
        print(f"총 GPU 메모리: {max_memory:.2f} GB")

# --- 데이터 로드 ---
print("데이터를 로드합니다...")
train_df_original = pd.read_csv('data/train.csv', encoding='utf-8-sig')
test_df = pd.read_csv('data/test.csv', encoding='utf-8-sig')
sample_submission = pd.read_csv('data/sample_submission.csv', encoding='utf-8-sig')
print("데이터 로드 완료.")

# # --- 딥러닝 모델 로드 (Mixed Precision 적용) ---
# print("딥러닝 모델들을 로드합니다...")
# bert_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
# bert_model = AutoModel.from_pretrained('klue/bert-base').half().to(device)  # FP16 적용
# print("모델 로드 완료.")
# print_gpu_utilization()


In [ ]:
# train_paragraph_df 재생성 (이상탐지 및 레이블에 필요)
print("📝 훈련 데이터 문단 분리 중...")
train_df_original['paragraphs'] = train_df_original['full_text'].str.split('\n')
train_paragraph_df = train_df_original.explode('paragraphs').rename(columns={'paragraphs': 'text'})
train_paragraph_df = train_paragraph_df.dropna(subset=['text'])
train_paragraph_df = train_paragraph_df[train_paragraph_df['text'].str.strip().astype(bool)].reset_index(drop=True)

print(f"✅ 문단 분리 완료: {len(train_paragraph_df):,}개")
print(f"사람 글: {(train_paragraph_df['generated'] == 0).sum():,}개")
print(f"AI 글: {(train_paragraph_df['generated'] == 1).sum():,}개")

In [ ]:
# --- 최적화된 특징 생성 함수 ---
def get_bert_embeddings_optimized(texts, batch_size=512):
    """
    A100 GPU 최적화된 BERT 임베딩 생성 함수
    - Mixed Precision (FP16) 적용
    - 대용량 배치 사이즈
    - 메모리 관리 최적화
    """
    all_embeddings = []

    # 메모리 사전 정리
    torch.cuda.empty_cache()
    gc.collect()

    print(f"배치 크기: {batch_size}, 총 텍스트 수: {len(texts)}")

    for i in tqdm(range(0, len(texts), batch_size), desc="BERT Embedding (최적화됨)"):
        try:
            batch = [str(t) for t in texts[i:i+batch_size]]

            # 토크나이징
            batch_dict = bert_tokenizer(
                batch,
                max_length=512,
                padding=True,
                truncation=True,
                return_tensors='pt'
            ).to(device)

            # Mixed Precision으로 Forward Pass
            with torch.no_grad():
                with torch.cuda.amp.autocast():
                    outputs = bert_model(**batch_dict)

            # 임베딩 추출 (float32로 변환하여 정확도 유지)
            embeddings = outputs.pooler_output.float()
            all_embeddings.append(embeddings.cpu().numpy())

            # 즉시 메모리 해제
            del batch_dict, outputs, embeddings

            # 주기적 메모리 정리 (10배치마다)
            if i % (10 * batch_size) == 0:
                torch.cuda.empty_cache()
                gc.collect()

        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"\n메모리 부족 발생! 배치 크기를 {batch_size//2}로 줄입니다.")
                torch.cuda.empty_cache()
                gc.collect()
                return get_bert_embeddings_optimized(texts, batch_size//2)
            else:
                raise e

    # 최종 메모리 정리
    torch.cuda.empty_cache()
    gc.collect()

    return np.vstack(all_embeddings)

def find_optimal_batch_size(sample_texts, start_batch=512, max_batch=2048):
    """최적 배치 크기 자동 탐색"""
    print("최적 배치 크기를 탐색합니다...")

    for batch_size in [start_batch, start_batch*2, max_batch]:
        try:
            print(f"배치 크기 {batch_size} 테스트 중...")
            # 소량 데이터로 테스트
            test_sample = sample_texts[:min(batch_size*2, len(sample_texts))]
            _ = get_bert_embeddings_optimized(test_sample, batch_size=batch_size)
            print(f"배치 크기 {batch_size} 성공!")
            optimal_batch = batch_size
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"배치 크기 {batch_size} 메모리 부족")
                break
            else:
                raise e

    torch.cuda.empty_cache()
    return optimal_batch


# 훈련 데이터 임베딩 + TF-IDF

In [ ]:
# # --- 2. 최적 배치 크기 찾기 ---
# sample_texts = train_paragraph_df['text'].head(1000).tolist()
# optimal_batch_size = find_optimal_batch_size(sample_texts)
# print(f"최적 배치 크기: {optimal_batch_size}")

# # --- 3. 훈련 데이터 모든 문단에 대한 특징 생성 (최적화됨) ---
# print(f"\n[훈련 문단 데이터] 특징 생성 시작... (배치 크기: {optimal_batch_size})")

# # 3-1. BERT 임베딩 (제목)
# print("제목 BERT 임베딩 생성 중...")
# print_gpu_utilization()
# title_embeddings = get_bert_embeddings_optimized(
#     train_paragraph_df['title'].tolist(),
#     batch_size=optimal_batch_size
# )
# print("제목 임베딩 완료!")
# print_gpu_utilization()

# # 3-2. BERT 임베딩 (본문)
# print("본문 BERT 임베딩 생성 중...")
# text_embeddings = get_bert_embeddings_optimized(
#     train_paragraph_df['text'].tolist(),
#     batch_size=optimal_batch_size
# )
# print("본문 임베딩 완료!")
# print_gpu_utilization()

# # 3-3. BERT 임베딩 결합
# X_train_p_emb = np.concatenate([title_embeddings, text_embeddings], axis=1)
# print(f"결합된 BERT 임베딩 shape: {X_train_p_emb.shape}")

# # 메모리 정리
# del title_embeddings, text_embeddings
# gc.collect()

# print("TF-IDF + SVD 특징 생성 중...")
# tfidf_pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=10000)),
#     ('svd', TruncatedSVD(n_components=128, random_state=42))
# ])
# X_train_p_tfidf = tfidf_pipeline.fit_transform(train_paragraph_df['text'])
# print(f"TF-IDF + SVD shape: {X_train_p_tfidf.shape}")

# # 3-5. 모든 특징 결합
# print("모든 특징을 결합합니다...")
# X_train_paragraph_features = np.concatenate([X_train_p_emb, X_train_p_tfidf], axis=1)
# print(f"최종 특징 행렬 shape: {X_train_paragraph_features.shape}")

# # --- 4. 결과 저장 ---
# print("특징 행렬을 저장합니다...")
# np.save('X_train_paragraph_features.npy', X_train_paragraph_features)
# print("저장 완료!")

# # --- 5. 최종 GPU 메모리 상태 ---
# print("\n=== 최종 GPU 메모리 상태 ===")
# print_gpu_utilization()

# # 메모리 정리
# torch.cuda.empty_cache()
# gc.collect()
# print("메모리 정리 완료!")

In [ ]:
# # 컬럼명 확인 및 변환 (필요시)
# if 'paragraph_text' in test_df.columns:
#     test_df = test_df.rename(columns={'paragraph_text': 'text'})
#     print("컬럼명 변환: paragraph_text → text")

# print(f"테스트 데이터 shape: {test_df.shape}")
# print(f"테스트 데이터 컬럼: {list(test_df.columns)}")

# # --- 2. 최적 배치 크기 찾기 (테스트 데이터 기준) ---
# sample_test_texts = test_df['text'].head(1000).tolist()
# optimal_batch_size = find_optimal_batch_size(sample_test_texts)
# print(f"테스트 데이터 최적 배치 크기: {optimal_batch_size}")

# # --- 3. 테스트 데이터 특징 생성 (최적화됨) ---
# print(f"\n[테스트 데이터] 특징 생성 시작... (배치 크기: {optimal_batch_size})")

# # 3-1. BERT 임베딩 (제목)
# print("테스트 데이터 제목 BERT 임베딩 생성 중...")
# print_gpu_utilization()
# test_title_embeddings = get_bert_embeddings_optimized(
#     test_df['title'].tolist(),
#     batch_size=optimal_batch_size
# )
# print("테스트 제목 임베딩 완료!")
# print_gpu_utilization()

# # 3-2. BERT 임베딩 (본문)
# print("테스트 데이터 본문 BERT 임베딩 생성 중...")
# test_text_embeddings = get_bert_embeddings_optimized(
#     test_df['text'].tolist(),
#     batch_size=optimal_batch_size
# )
# print("테스트 본문 임베딩 완료!")
# print_gpu_utilization()

# # 3-3. BERT 임베딩 결합
# X_test_p_emb = np.concatenate([test_title_embeddings, test_text_embeddings], axis=1)
# print(f"테스트 결합된 BERT 임베딩 shape: {X_test_p_emb.shape}")

# # 메모리 정리
# del test_title_embeddings, test_text_embeddings
# gc.collect()

# # --- 3-4. TF-IDF + SVD 처리 ---
# print("테스트 데이터에 TF-IDF transform 적용...")
# X_test_p_tfidf = tfidf_pipeline.transform(test_df['text'])
# print(f"테스트 TF-IDF + SVD shape: {X_test_p_tfidf.shape}")

# # --- 3-5. 모든 특징 결합 ---
# print("테스트 데이터의 모든 특징을 결합합니다...")
# X_test_paragraph_features = np.concatenate([X_test_p_emb, X_test_p_tfidf], axis=1)
# print(f"테스트 최종 특징 행렬 shape: {X_test_paragraph_features.shape}")

# # --- 4. 결과 저장 ---
# print("\n테스트 특징 행렬을 저장합니다...")
# np.save('X_test_paragraph_features.npy', X_test_paragraph_features)
# print("✅ X_test_paragraph_features.npy 저장 완료!")

In [16]:
# --- 미리 생성된 특징(.npy) 파일 로드 ---
print("💾 미리 생성된 훈련 데이터 특징 파일을 로드합니다...")

# data 폴더에 저장된 .npy 파일 경로를 사용합니다.
X_train_paragraph_features = np.load('data/X_train_paragraph_features.npy')

print(f"✅ 훈련 특징 로드 완료: {X_train_paragraph_features.shape}")

✅ 훈련 특징 로드 완료: (1226364, 1664)


In [17]:
print("🔍 하이브리드 이상탐지 시작")
start_time = time.time()

# 사람 글 인덱스 추출 (라벨 0)
human_indices = np.where(train_paragraph_df['generated'] == 0)[0]
print(f"사람 글 문단: {len(human_indices):,}개")

# 1단계: Isolation Forest (빠른 1차 필터링)
print("  1단계: Isolation Forest...")
iso_forest = IsolationForest(
    contamination=0.15,       # 15% 이상치 가정
    n_estimators=200,
    max_samples='auto',
    random_state=42,
    n_jobs=-1
)

iso_outliers = iso_forest.fit_predict(X_train_paragraph_features[human_indices])
stage1_clean = human_indices[iso_outliers == 1]
print(f"  1차 필터링: {len(stage1_clean):,}개 남음")

# 2단계: LOF (정교한 2차 필터링)
print("  2단계: LOF...")
lof = LocalOutlierFactor(
    n_neighbors=min(50, len(stage1_clean) // 20),
    contamination=0.05,       # 5% 추가 제거
    algorithm='auto',
    n_jobs=-1
)

lof_outliers = lof.fit_predict(X_train_paragraph_features[stage1_clean])
final_clean_indices = stage1_clean[lof_outliers == 1]

elapsed = time.time() - start_time
removed = len(human_indices) - len(final_clean_indices)

print(f"✅ 하이브리드 이상탐지 완료! ({elapsed:.1f}초)")
print(f"  제거된 노이즈: {removed:,}개 ({removed/len(human_indices)*100:.1f}%)")
print(f"  정제된 사람 글: {len(final_clean_indices):,}개")

# 정제된 훈련 데이터 구성
# 정제된 사람 글 + 모든 AI 글
ai_indices = np.where(train_paragraph_df['generated'] == 1)[0]
clean_indices = np.concatenate([final_clean_indices, ai_indices])

X_train_clean = X_train_paragraph_features[clean_indices]
y_train_clean = train_paragraph_df['generated'].iloc[clean_indices].values

print(f"\n📊 정제된 훈련 데이터: {X_train_clean.shape}")
print(f"  정제된 사람 글: {(y_train_clean == 0).sum():,}개")
print(f"  AI 글: {(y_train_clean == 1).sum():,}개")

🔍 하이브리드 이상탐지 시작
사람 글 문단: 1,125,652개
  1단계: Isolation Forest...
  1차 필터링: 956,804개 남음
  2단계: LOF...
✅ 하이브리드 이상탐지 완료! (12734.1초)
  제거된 노이즈: 216,689개 (19.3%)
  정제된 사람 글: 908,963개

📊 정제된 훈련 데이터: (1009675, 1664)
  정제된 사람 글: 908,963개
  AI 글: 100,712개


In [19]:
train_paragraph_df.head(20)

,title,full_text,generated,text
0,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...
1,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에...
2,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. ...
3,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로...
4,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬...
5,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,1941년 12월 7일에 일어난 일본 제국 해군의 진주만 공격을 계기로 카호올라웨...
6,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은..."
7,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0,"용어는 각자 다른 진화 단계에 있는 여러 가지 별에 적용되는데, 이들 모두 주계열..."
8,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0,"청색거성이라는 명칭은 종종 매우 크고 뜨거운 주계열성과 같이, 다른 무겁고 밝은 ..."
9,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0,청색거성은 엄격히 정의된 단어가 아니어서 서로 다른 다양한 유형의 별에 폭넓게 사...


In [20]:
print("🎯 StratifiedGroupKFold LightGBM 학습 시작 (데이터 누수 방지)")

# Optuna 최적화된 파라미터
best_params = {
    'learning_rate': 0.043,
    'num_leaves': 41,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.84,
    'lambda_l1': 9.28e-06,
    'lambda_l2': 0.0021,
    'objective': 'binary',
    'metric': 'auc',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'random_state': 42,
    'n_estimators': 1000,
    'device': 'cpu',  # GPU → CPU 변경
    'n_jobs': 16
}

# --- 그룹 교차검증 설정 ---
# 1. 그룹 정보 생성 (원본 글 ID 기준)
# clean_indices는 정제된 문단들의 인덱스입니다.
# 이 인덱스를 사용하여 원본 train_paragraph_df에서 글 ID('id')를 가져와 그룹으로 사용합니다.
groups = train_paragraph_df['title'].iloc[clean_indices].values
print(f"그룹 교차검증을 위한 그룹 생성 완료: {len(np.unique(groups))}개 고유 글")

# 2. StratifiedGroupKFold 설정
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

# OOF 예측을 위한 배열 초기화
oof_predictions = np.zeros(len(X_train_clean))
fold_models = []
fold_scores = []

print(f"\n🔄 5-Fold Group 교차검증 시작...")

# 3. 교차검증 루프 변경
# split 메서드에 groups 정보를 추가합니다.
for fold, (train_idx, val_idx) in enumerate(sgkf.split(X_train_clean, y_train_clean, groups)):
    print(f"\n📁 Fold {fold + 1}/5")
    
    X_fold_train, X_fold_val = X_train_clean[train_idx], X_train_clean[val_idx]
    y_fold_train, y_fold_val = y_train_clean[train_idx], y_train_clean[val_idx]
    
    # 클래스 분포 확인
    train_ratio = y_fold_train.mean()
    val_ratio = y_fold_val.mean()
    print(f"  훈련 세트 AI 비율: {train_ratio:.3f}")
    print(f"  검증 세트 AI 비율: {val_ratio:.3f}")
    
    # LightGBM 데이터셋 생성
    train_dataset = lgb.Dataset(X_fold_train, label=y_fold_train)
    val_dataset = lgb.Dataset(X_fold_val, label=y_fold_val, reference=train_dataset)
    
    # 모델 학습
    fold_model = lgb.train(
        best_params,
        train_dataset,
        valid_sets=[val_dataset],
        valid_names=['valid'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(period=0)  # 로그 출력 최소화
        ]
    )
    
    # 검증 세트 예측
    val_pred = fold_model.predict(X_fold_val, num_iteration=fold_model.best_iteration)
    oof_predictions[val_idx] = val_pred
    
    # AUC 점수 계산
    fold_auc = roc_auc_score(y_fold_val, val_pred)
    fold_scores.append(fold_auc)
    fold_models.append(fold_model)
    
    print(f"  Fold {fold + 1} AUC: {fold_auc:.5f}")

# 전체 OOF AUC 계산
oof_auc = roc_auc_score(y_train_clean, oof_predictions)
mean_auc = np.mean(fold_scores)
std_auc = np.std(fold_scores)

print(f"\n🏆 교차검증 결과 (데이터 누수 방지됨):")
print(f"  평균 AUC: {mean_auc:.5f} (±{std_auc:.5f})")
print(f"  OOF AUC: {oof_auc:.5f}")
print(f"  최고 Fold AUC: {max(fold_scores):.5f}")
print(f"  최저 Fold AUC: {min(fold_scores):.5f}")


🎯 StratifiedGroupKFold LightGBM 학습 시작 (데이터 누수 방지)
그룹 교차검증을 위한 그룹 생성 완료: 92861개 고유 글

🔄 5-Fold Group 교차검증 시작...

📁 Fold 1/5
  훈련 세트 AI 비율: 0.100
  검증 세트 AI 비율: 0.099
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[78]	valid's auc: 0.712611
  Fold 1 AUC: 0.71261

📁 Fold 2/5
  훈련 세트 AI 비율: 0.100
  검증 세트 AI 비율: 0.100
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[159]	valid's auc: 0.709926
  Fold 2 AUC: 0.70993

📁 Fold 3/5
  훈련 세트 AI 비율: 0.100
  검증 세트 AI 비율: 0.100
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[332]	valid's auc: 0.72295
  Fold 3 AUC: 0.72295

📁 Fold 4/5
  훈련 세트 AI 비율: 0.101
  검증 세트 AI 비율: 0.096
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	valid's auc: 0.712711
  Fold 4 AUC: 0.71271

📁 Fold 5/5
  훈련 세트 AI 비율: 0.099
  검증 세트 AI 비율: 0.104
Training until validation scores d

In [22]:
print("🔮 테스트 데이터 최종 예측 및 제출 파일 생성")

# 5개 폴드 모델의 평균 예측
print("  📊 앙상블 예측 수행 중...")
X_test_features = np.load('data/X_test_paragraph_features.npy')
print(f"✅ 테스트 특징 로드 완료: {X_test_features.shape}")
test_predictions_list = []

if 'fold_models' not in locals():
    print("❌ fold_models가 정의되지 않았습니다.")
    print("StratifiedKFold 학습 셀을 먼저 실행해주세요.")

for i, model in enumerate(fold_models):
    fold_pred = model.predict(X_test_features, num_iteration=model.best_iteration)
    test_predictions_list.append(fold_pred)
    print(f"  Fold {i+1} 예측 완료")

# 평균 앙상블
ensemble_predictions = np.mean(test_predictions_list, axis=0)

# 제출 파일 생성
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission['generated'] = ensemble_predictions

# 결과 폴더 생성 및 파일 저장
os.makedirs('results', exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
submission_filename = f'results/submission_hybrid_lgb_{timestamp}.csv'

sample_submission.to_csv(submission_filename, index=False)

# 기본 제출 파일도 생성
sample_submission.to_csv('submission.csv', index=False)

print(f"\n🎉 제출 파일 생성 완료:")
print(f"  타임스탬프 파일: {submission_filename}")
print(f"  기본 파일: submission.csv")
print(f"  샘플 수: {len(sample_submission):,}개")

# 최종 결과 요약
print(f"\n📋 최종 파이프라인 완료:")
print(f"  🎯 StratifiedKFold 학습: 5-Fold CV 완료")
print(f"  🏆 최종 OOF AUC: {oof_auc:.5f}")
print(f"  🔮 앙상블 예측: 5개 모델 평균")
print(f"  ⏰ 완료 시간: {datetime.now()}")
print("✅ 모든 작업 완료! 제출 파일을 대회에 업로드하세요.")


🔮 테스트 데이터 최종 예측 및 제출 파일 생성
  📊 앙상블 예측 수행 중...
✅ 테스트 특징 로드 완료: (1962, 1664)
  Fold 1 예측 완료
  Fold 2 예측 완료
  Fold 3 예측 완료
  Fold 4 예측 완료
  Fold 5 예측 완료

🎉 제출 파일 생성 완료:
  타임스탬프 파일: results/submission_hybrid_lgb_20250714_0522.csv
  기본 파일: submission.csv
  샘플 수: 1,962개

📋 최종 파이프라인 완료:
  🎯 StratifiedKFold 학습: 5-Fold CV 완료
  🏆 최종 OOF AUC: 0.71047
  🔮 앙상블 예측: 5개 모델 평균
  ⏰ 완료 시간: 2025-07-14 05:22:53.046723
✅ 모든 작업 완료! 제출 파일을 대회에 업로드하세요.
